In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform


/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")




In [3]:
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")


In [4]:
baseline_aca_enrollment = baseline.calculate("has_marketplace_health_coverage", map_to="person", period=2026).sum()

reformed_aca_enrollment = reformed.calculate("has_marketplace_health_coverage", map_to="person", period=2026).sum()
difference_aca_enrollment = reformed_aca_enrollment - baseline_aca_enrollment

In [17]:
baseline_has_coverage = baseline.calculate("has_marketplace_health_coverage", map_to="person", period=2026)
baseline_is_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="person", period=2026)

baseline_aca_enrollment = ((baseline_has_coverage & baseline_is_eligible)*baseline_has_coverage.weights).sum()

baseline_is_eligible.sum()/1e6

40.21955536317878

In [6]:
baseline_is_eligible.sum()

40219555.363178775

In [7]:
baseline_has_coverage.sum()

20197518.476335302

In [8]:
reform_has_coverage = reformed.calculate("has_marketplace_health_coverage", map_to="person", period=2026)
reform_is_eligible = reformed.calculate("is_aca_ptc_eligible", map_to="person", period=2026)

reform_aca_enrollment = ((reform_has_coverage & reform_is_eligible)*reform_has_coverage.weights).sum()

reform_aca_enrollment.sum()

12432543.975246934

In [9]:
reform_has_coverage.sum()

20197518.476335302

In [10]:
reform_is_eligible.sum()

76212421.75032558

In [11]:
baseline_slcsp = baseline.calculate("aca_ptc", map_to="household", period=2026)
baseline_slcsp.sum()

109008121808.43225

In [12]:
year=2026
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)


In [13]:
aca_baseline = baseline.calculate("aca_ptc", map_to="household", period=year)
aca_reform =  reformed.calculate("aca_ptc", map_to="household", period=year)
baseline_aca_enrollment.sum()

6155068.245171301

In [14]:
import pandas as pd

# Create a DataFrame with the outputs
data = {
    "State": state,
    "Married": married,
    "Num_Dependents": num_dependents,
    "Employment_Income": employment_income,
    "Self_Employment_Income": self_employment_income,
    "ACA_Baseline": aca_baseline,
    "ACA_Reform": aca_reform,
}

df_outputs = pd.DataFrame(data)
df_outputs.head()  # Display the first few rows of the DataFrame

,State,Married,Num_Dependents,Employment_Income,Self_Employment_Income,ACA_Baseline,ACA_Reform
0,ME,1.0,0.0,4412.632324,0.0,0.0,0.0
1,ME,1.0,0.0,101122.818359,0.0,0.0,0.0
2,ME,0.0,0.0,0.000000,0.0,0.0,0.0
3,ME,0.0,0.0,91929.835938,0.0,0.0,0.0
4,ME,0.0,0.0,36588.075684,0.0,0.0,0.0


In [26]:
# Get household weights
household_weights = baseline.calculate("household_weight", map_to="household", period=year)

# Add weights to the dataframe
df_outputs["Household_Weight"] = household_weights

# Filter for households with aca_ptc > 0 (using baseline ACA)
households_with_aca = df_outputs[df_outputs["ACA_Baseline"] > 0]

# Sort by household weight (descending) and get top 10
top_10_highest_weight = households_with_aca.nlargest(10, "Household_Weight")

print("Top 10 highest weight households with ACA PTC > 0:")
print(top_10_highest_weight[["State", "Married", "Num_Dependents", "Employment_Income", 
                            "Self_Employment_Income", "ACA_Baseline", "ACA_Reform", "Household_Weight"]])

Top 10 highest weight households with ACA PTC > 0:


KeyError: "['Age'] not in index"